# Volumes

# Background

# Types of volumes 

### emptyDir

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: test-pd
spec:
  containers:
  - image: registry.k8s.io/test-webserver
    name: test-container
    volumeMounts:
    - mountPath: /cache
      name: cache-volume
  volumes:
  - name: cache-volume
    emptyDir: {}

---

### 範例

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: multicontainer-pod
spec:
  containers:
  - name: producer
    image: busybox
    command: ["sh", "-c", "while true; do echo $(hostname) $(date) >> /var/log/index.html; sleep 10; done"]
    volumeMounts:
    - name: webcontent
      mountPath: /var/log
  - name: consumer
    image: nginx
    ports:
      - containerPort: 80
    volumeMounts:
    - name: webcontent
      mountPath: /usr/share/nginx/html
  volumes:
  - name: webcontent
    emptyDir: {}

### hostPath

<img src='./img/30.png'>

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: test-pd
spec:
  containers:
  - image: registry.k8s.io/test-webserver
    name: test-container
    volumeMounts:
    - mountPath: /test-pd
      name: test-volume
  volumes:
  - name: test-volume
    hostPath:
      # directory location on host
      path: /data
      # this field is optional
      type: Directory

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: test-webserver
spec:
  containers:
  - name: test-webserver
    image: registry.k8s.io/test-webserver:latest
    volumeMounts:
    - mountPath: /var/local/aaa
      name: mydir
    - mountPath: /var/local/aaa/1.txt
      name: myfile
  volumes:
  - name: mydir
    hostPath:
      # Ensure the file directory is created.
      path: /var/local/aaa
      type: DirectoryOrCreate
  - name: myfile
    hostPath:
      path: /var/local/aaa/1.txt
      type: FileOrCreate

### 範例

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: multicontainer-pod
spec:
  containers:
  - name: producer
    image: busybox
    command: ["sh", "-c", "while true; do echo $(hostname) $(date) >> /var/log/index.html; sleep 10; done"]
    volumeMounts:
    - name: webcontent
      mountPath: /var/log
  - name: consumer
    image: nginx
    ports:
      - containerPort: 80
    volumeMounts:
    - name: webcontent
      mountPath: /usr/share/nginx/html
  volumes:
  - name: webcontent
    hostPath:
      path: /tmp
      type: Directory

### local

In [ ]:
apiVersion: v1
kind: PersistentVolume
metadata:
  name: example-pv
spec:
  capacity:
    storage: 100Gi
  volumeMode: Filesystem
  accessModes:
  - ReadWriteOnce
  persistentVolumeReclaimPolicy: Delete
  storageClassName: local-storage
  local:
    path: /mnt/disks/ssd1
  nodeAffinity:
    required:
      nodeSelectorTerms:
      - matchExpressions:
        - key: kubernetes.io/hostname
          operator: In
          values:
          - example-node

In [ ]:
待補

### nfs

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: test-pd
spec:
  containers:
  - image: registry.k8s.io/test-webserver
    name: test-container
    volumeMounts:
    - mountPath: /my-nfs-data
      name: test-volume
  volumes:
  - name: test-volume
    nfs:
      server: my-nfs-server.example.com
      path: /my-nfs-volume
      readOnly: true

# Persistent Volumes

### Introduction

# Lifecycle of a volume and claim

### Provisioning

##### Static

#### Dynamic

### Binding

### Using

### Reclaiming

##### Retain

##### Delete

### 範例

In [ ]:
apiVersion: v1
kind: PersistentVolume
metadata:
  name: pv-nfs
spec:
  storageClassName: standard
  capacity:
    storage: 4Gi
  accessModes:
    - ReadWriteMany
  persistentVolumeReclaimPolicy: Retain
  nfs:
    server: 20.243.250.224
    path: "/export/volumes/pod"

In [ ]:
kubectl apply -f pv.yml 

In [ ]:
kubectl get pv

In [ ]:
kubectl describe pv pv-nfs

---

In [ ]:
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: pvc-nfs
spec:
  storageClassName: standard
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 1Gi

In [ ]:
kubectl apply -f pvc.yml

In [ ]:
kubectl get pvc 

In [ ]:
kubectl describe pvc pvc-nfs

---

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: web
spec:
  replicas: 1
  selector:
    matchLabels:
      app: web
  template:
    metadata:
      labels:
        app: web
    spec:
      volumes:
      - name: webcontent
        persistentVolumeClaim:
          claimName: pvc-nfs
      containers:
      - image: nginx
        name: nginx
        ports:
        - containerPort: 80
        volumeMounts:
        - name: webcontent
          mountPath: "/usr/share/nginx/html/web-app"

In [ ]:
kubectl apply -f nginx.yml 

---

In [ ]:
vagrant@nfs-server:/export/volumes/pod$ pwd
/export/volumes/pod
vagrant@nfs-server:/export/volumes/pod$ ls
index.html
vagrant@nfs-server:/export/volumes/pod$ more index.html
hello k8s
vagrant@nfs-server:/export/volumes/pod$

---

In [ ]:
kubectl expose deployment web --port=80 --type=NodePort

In [ ]:
kubectl get service

In [ ]:
minikube service web --url

---

In [ ]:
kubectl delete service web
kubectl delete deployments.apps web
kubectl delete persistentvolumeclaims pvc-nfs
kubectl delete persistentvolume pv-nfs

### 範例: Configure a Pod to Use a PersistentVolume for Storage

##### Before you begin

##### Create an index.html file on your Node

In [ ]:
# This assumes that your Node uses "sudo" to run commands
# as the superuser
sudo mkdir /mnt/data

In [ ]:
# This again assumes that your Node uses "sudo" to run commands
# as the superuser
sudo sh -c "echo 'Hello from Kubernetes storage' > /mnt/data/index.html"

In [ ]:
cat /mnt/data/index.html

##### Create a PersistentVolume

In [ ]:
apiVersion: v1
kind: PersistentVolume
metadata:
  name: task-pv-volume
  labels:
    type: local
spec:
  storageClassName: manual
  capacity:
    storage: 10Gi
  accessModes:
    - ReadWriteOnce
  hostPath:
    path: "/mnt/data"

In [ ]:
kubectl apply -f https://k8s.io/examples/pods/storage/pv-volume.yaml

In [ ]:
kubectl get pv task-pv-volume

##### Create a PersistentVolumeClaim

In [ ]:
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: task-pv-claim
spec:
  storageClassName: manual
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 3Gi

In [ ]:
kubectl apply -f https://k8s.io/examples/pods/storage/pv-claim.yaml

In [ ]:
kubectl get pv task-pv-volume

In [ ]:
kubectl get pvc task-pv-claim

##### Create a Pod

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: task-pv-pod
spec:
  volumes:
    - name: task-pv-storage
      persistentVolumeClaim:
        claimName: task-pv-claim
  containers:
    - name: task-pv-container
      image: nginx
      ports:
        - containerPort: 80
          name: "http-server"
      volumeMounts:
        - mountPath: "/usr/share/nginx/html"
          name: task-pv-storage

In [ ]:
kubectl apply -f https://k8s.io/examples/pods/storage/pv-pod.yaml

In [ ]:
kubectl get pod task-pv-pod

In [ ]:
kubectl exec -it task-pv-pod -- /bin/bash

In [ ]:
# Be sure to run these 3 commands inside the root shell that comes from
# running "kubectl exec" in the previous step
apt update
apt install curl
curl http://localhost/

In [ ]:
If you see that message, you have successfully configured a Pod to use storage from a PersistentVolumeClaim.

##### Clean up

In [ ]:
kubectl delete pod task-pv-pod
kubectl delete pvc task-pv-claim
kubectl delete pv task-pv-volume

In [ ]:
# This assumes that your Node uses "sudo" to run commands
# as the superuser
sudo rm /mnt/data/index.html
sudo rmdir /mnt/data

##### Mounting the same persistentVolume in two places

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: test
spec:
  containers:
    - name: test
      image: nginx
      volumeMounts:
        # a mount for site-data
        - name: config
          mountPath: /usr/share/nginx/html
          subPath: html
        # another mount for nginx config
        - name: config
          mountPath: /etc/nginx/nginx.conf
          subPath: nginx.conf
  volumes:
    - name: config
      persistentVolumeClaim:
        claimName: task-nfs-claim

### Access control

In [ ]:
apiVersion: v1
kind: PersistentVolume
metadata:
  name: pv1
  annotations:
    pv.beta.kubernetes.io/gid: "1234"